In [1]:
import torchvision
import torchvision.transforms as transforms

c:\Users\sergi\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] No se encontró el proceso especificado'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
my_tranform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

dataset = torchvision.datasets.ImageFolder(root='Shapes', transform=my_tranform)

Circles -> 0
Stars -> 1
Triangles -> 2

In [3]:
import torch
import torch.utils.data as data_utils

In [4]:
dataset[0][0].shape

torch.Size([3, 64, 64])

In [5]:
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size = 64, shuffle = True)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [8]:
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, \
                                             batch_size=64,
                                             shuffle=True)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset, \
                                             batch_size=64)

In [17]:
nb_neurons = 64-5-5-5-5+4
nb_kernel = 64

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.layer1 = nn.Conv2d(3, 8, 5)
        # self.layer2 = nn.Conv2d(8, 16, 5)
        # self.layer3 = nn.Conv2d(16, 32, 5)
        # self.layer4 = nn.Conv2d(32, 64, 5)
        # self.layer5 = nn.Linear((nb_neurons**2)*nb_kernel, 3)
        self.layer1 = nn.Conv2d(3, 8, 5, padding = 2)
        self.layer2 = nn.Conv2d(8, 16, 5, padding = 2)
        self.layer3 = nn.Conv2d(16, 32, 5, padding = 2)
        self.layer4 = nn.Conv2d(32, 64, 5, padding = 2)
        self.layer5 = nn.Linear(64*64*64, 3)
        
        self.relu = nn.ReLU()
    def forward(self, x):
        l = self.relu(self.layer1(x))
        l = self.relu(self.layer2(l))
        l = self.relu(self.layer3(l))
        l = self.relu(self.layer4(l))
        l = l.view(-1,64*64*64)
        # l = l.view(-1,(nb_neurons**2)*nb_kernel)
        return F.softmax(self.layer5(l), dim=1)

net = Net()
opt = optim.Adam(net.parameters(), 0.0001)
loss_fn = nn.MSELoss()
losses = []
accuracies = [0.1]

for epoch in range(100):
    total_correct = 0
    total = train_size
    for images, labels in train_dataset_loader:
        print(images.max(), images.min(), images.mean(), images.std())
        hot_predictions = net(images)
        predictions = hot_predictions.argmax(axis=1)
        nb_correct = sum(predictions == labels)
        total_correct += nb_correct
        hot_labels = F.one_hot(labels, 3).to(torch.float32)
        loss = loss_fn(hot_labels, hot_predictions)
        loss.backward()
        losses.append(loss.item())
        opt.step()
        opt.zero_grad()
    acc = total_correct/total
    accuracies.append(acc.item())
    print(f"Epoch: {epoch}\t Accuracy: {acc}")

plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(losses)
plt.subplot(1,2,2)
plt.plot(accuracies, "*r")
plt.ylim(0,1)
plt.show()

tensor(1.) tensor(0.) tensor(0.6946) tensor(0.1746)


c:\Users\sergi\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


tensor(1.) tensor(0.) tensor(0.7189) tensor(0.1918)
tensor(1.) tensor(0.) tensor(0.7207) tensor(0.1667)
tensor(1.) tensor(0.) tensor(0.7178) tensor(0.1788)
tensor(1.) tensor(0.) tensor(0.7269) tensor(0.1968)
tensor(1.) tensor(0.) tensor(0.7156) tensor(0.1630)
tensor(1.) tensor(0.) tensor(0.7138) tensor(0.1740)
tensor(1.) tensor(0.) tensor(0.7247) tensor(0.1767)
Epoch: 0	 Accuracy: 0.35729387402534485
tensor(1.) tensor(0.) tensor(0.7296) tensor(0.1894)
tensor(1.) tensor(0.) tensor(0.7233) tensor(0.1772)
tensor(1.) tensor(0.) tensor(0.7280) tensor(0.1783)
tensor(1.) tensor(0.) tensor(0.6812) tensor(0.1754)
tensor(1.) tensor(0.) tensor(0.7254) tensor(0.1801)
tensor(1.) tensor(0.) tensor(0.7066) tensor(0.1633)
tensor(1.) tensor(0.) tensor(0.7032) tensor(0.1757)
tensor(1.) tensor(0.) tensor(0.7527) tensor(0.1838)
Epoch: 1	 Accuracy: 0.37632134556770325
tensor(1.) tensor(0.) tensor(0.7069) tensor(0.1710)
tensor(1.) tensor(0.) tensor(0.7253) tensor(0.1798)
tensor(1.) tensor(0.) tensor(0.7135)

KeyboardInterrupt: 

# Test

In [11]:
total_correct = 0
total = test_size
for images, labels in test_dataset_loader:
    hot_predictions = net(images)
    predictions = hot_predictions.argmax(axis=1)
    nb_correct = sum(predictions == labels)
    total_correct += nb_correct
    hot_labels = F.one_hot(labels, 3).to(torch.float32)
    loss = loss_fn(hot_labels, hot_predictions)
    loss.backward()
    opt.step()
    opt.zero_grad()
acc = total_correct/total
print(f"Accuracy: {acc}")

Accuracy: 0.9495798349380493


In [13]:
images.shape

torch.Size([55, 3, 64, 64])

In [14]:
dummy_input = torch.ones(1, 3, 64, 64)

In [15]:
#Saving the model in a onnx file
torch.onnx.export(net, dummy_input, 'Test_model.onnx', export_params = True, verbose = True)